In [ ]:
import ipywidgets.widgets as widgets
import sys
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.patches as patches
from matplotlib.transforms import Affine2D
from io import BytesIO

from ballsbot_cpp import ballsbot_cpp

sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.utils import figsize_from_image_size, keep_rps

In [ ]:
def show_detections(image, detections):
    fig = Figure(figsize=figsize_from_image_size(image))
    canvas = FigureCanvas(fig)
    ax = fig.gca()

    for det in detections:
        x0 = det['bottom_left'][0]
        w = det['top_right'][0] - det['bottom_left'][0]

        y0 = det['bottom_left'][1]
        h = det['top_right'][1] - det['bottom_left'][1]

        rect = patches.Rectangle(
            (x0, y0), w, h,
            linewidth=2, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)

        ax.text(
            x0, y0,
            '{} ({:0.0f}%)'.format(det['object_class'], det['confidence'] * 100.),
            fontsize=10, color='r'
        )

    ax.set_xlim(0., 1.)
    ax.set_ylim(0., 1.)
    ax.grid(which='both', linestyle='--', alpha=0.5)

    canvas.draw()
    jpeg = BytesIO()
    canvas.print_jpg(jpeg)
    image.value = jpeg.getvalue()

In [ ]:
image = widgets.Image(format='jpeg', width=600, height=600)
display(image)

In [ ]:
%%time
ballsbot_cpp.startup_detection()

In [ ]:
ts = None
while True:
    ts = keep_rps(ts, fps=2)
    detections = ballsbot_cpp.detect()
    show_detections(image, detections)